In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
from scipy.stats import norm,multivariate_normal

import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA

import torch
from torchvision import transforms


In [18]:
path = 'training.pt'
features_, labels = torch.load(path)

trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(14),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

features = torch.zeros((len(features_), 14, 14))
for i in range(len(features_)):
    features[i] = trans(features_[i])

features = features.numpy().reshape(-1, 14 * 14)[:30000]
labels = labels.numpy()[:30000]

xy = features
label = labels

n_class = 10
n_dim = 14*14







In [19]:
# p = [0.9,0.8,0.7,0.6,0.5,0.5,0.6,0.7,0.8,0.9]
p = []
for i in range(n_class):
    p.append(np.random.uniform(0.3,0.7))


p_eval = []
for i in range(n_class):
    p_eval.append(np.random.uniform(0.6,0.7))


# p_eval = [0.6,0.7,0.6,0.7,0.6,0.7,0.6,0.7,0.6,0.7]

policy_dict = {}
for i in range(n_class):
    policy_dict[i] = []
    for t in range(n_class):
        if t == i:
            policy_dict[i].append(p[i])
        else:
            policy_dict[i].append((1-p[i])/n_class)

eval_policy_dict = {}
for i in range(n_class):
    eval_policy_dict[i] = []
    for t in range(n_class):
        if t == i:
            eval_policy_dict[i].append(p_eval[i])
        else:
            eval_policy_dict[i].append((1-p_eval[i])/n_class)



# eval_policy_dict = {
#     0:[0.4,0.6/3,0.6/3,0.6/3],
#     1:[0.5/3,0.5,0.5/3,0.5/3],
#     2:[0.1/3,0.1/3,0.9,0.1/3],
#     3:[0.7/3,0.7/3,0.7/3,0.3],
# }
# a = 0.8
# b = 20
# size = 5000

# mean_logging = [0.,0.8]
# covariance_logging = [[0.1,0],[0,0.1]]

# mean_eval = [0.5,0.5]
# covariance_eval = [[10,0],[0,10]]
# vis_gaussian(mean_logging,covariance_logging)


In [20]:
def get_distribution(xy,a,b,pca_model=None):
    if not pca_model:
        pca_model = PCA(n_components=1).fit(xy)
    first_component = pca_model.transform(xy)
    m = np.min(first_component)
    m_bar = np.mean(first_component)
    mean = m + (m_bar-m)/a
    std = np.sqrt((m_bar - m)/b)
    pdf = norm(loc = mean,scale = std).pdf(first_component)
    return first_component, pca_model, pdf, mean, std

def get_gaussian_distribution(xy,mean,covariance):
    pdf = multivariate_normal(mean, covariance).pdf(xy)
    return pdf



In [21]:
def take_action(true_label, policy):
    action = random.choices([t for t in range(n_class)],policy[true_label])[0]
    prob = policy[true_label][action]
    return action, prob
    
    

In [22]:
def sample_logging_data(xy,first_component,label,size,pdf,pdf2,mean_eval,cov_eval):
    sampled_xy = np.zeros((size,n_dim))
    sampled_component = np.zeros(size)
    sampled_ture_label = np.zeros(size)
    sampled_action = np.zeros(size)
    
    sampled_xshift_prob = np.zeros(size)
    sampled_xshift_prob_for_eval = np.zeros(size)
    sampled_policy_prob = np.zeros(size)
    sampled_eval_policy_prob = np.zeros(size)
    
    
    
    sampled_reward = np.zeros(size)

    index_list = [i for i in range(len(xy))]
    for i in range(size):
        index = random.choices(index_list,weights=pdf)[0]
        sampled_xshift_prob[i] = pdf[index]
        sampled_xy[i] = xy[index]
        sampled_component[i] = first_component[index]
        sampled_ture_label[i] = label[index]
        sampled_action[i],sampled_policy_prob[i] = take_action(label[index], policy_dict)
        sampled_eval_policy_prob[i] = eval_policy_dict[label[index]][int(sampled_action[i])]
        sampled_xshift_prob_for_eval[i] = norm(loc = mean_eval,scale = cov_eval).pdf(first_component[index])
        
        
        sampled_reward[i] = int(sampled_action[i] == label[index])
    sampled_xshift_prob_for_eval = sampled_xshift_prob_for_eval/sum(pdf2)
        
    return sampled_xy,first_component, sampled_action, sampled_reward, sampled_ture_label, sampled_policy_prob,sampled_eval_policy_prob, sampled_xshift_prob,sampled_xshift_prob_for_eval
        
        
        
        
    

In [23]:
def sample_data(xy,first_component,label,size,pdf,pdf2,mean_logging,cov_logging):
    sampled_xy = np.zeros((size,n_dim))
    sampled_ture_label = np.zeros(size)
    sampled_action = np.zeros(size)
    sampled_component = np.zeros(size)
    
    sampled_xshift_prob = np.zeros(size)
    sampled_xshift_prob_for_logging = np.zeros(size)
    sampled_policy_prob = np.zeros(size)
    sampled_eval_policy_prob = np.zeros(size)
    
    sampled_reward = np.zeros(size)

    index_list = [i for i in range(len(xy))]
    for i in range(size):
        index = random.choices(index_list,weights=pdf2)[0]
        sampled_xshift_prob[i] = pdf2[index]
        sampled_xy[i] = xy[index]
        sampled_component[i] = first_component[index]
        sampled_ture_label[i] = label[index]
        sampled_action[i],sampled_policy_prob[i] = take_action(label[index], policy_dict)
        sampled_eval_policy_prob[i] = eval_policy_dict[label[index]][int(sampled_action[i])]
        sampled_xshift_prob_for_logging[i] = norm(loc = mean_logging,scale = cov_logging).pdf(first_component[index])
        
        
        sampled_reward[i] = int(sampled_action[i] == label[index])
    sampled_xshift_prob_for_logging = sampled_xshift_prob_for_logging/sum(pdf)
    return sampled_xy,sampled_component, sampled_action, sampled_reward, sampled_ture_label, sampled_policy_prob,sampled_eval_policy_prob, sampled_xshift_prob, sampled_xshift_prob_for_logging
        
        

In [24]:
def p_xshift(train_x,test_x):
    from sklearn.linear_model import LogisticRegression
    X = np.concatenate((train_x,test_x),axis = 0)
#     X = np.sum(X,axis = 1,keepdims = True)
    Y = np.concatenate((np.ones(len(train_x)),np.zeros(len(test_x))),axis = 0)
    clf = LogisticRegression().fit(X,Y)
    return clf

def loggingpolicy(x,y):
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression().fit(x,y)
    return clf

In [25]:
def evaluate(sampled_xy, sampled_action, sampled_reward, \
             sampled_ture_label, sampled_policy_prob,\
             sampled_eval_policy_prob, sampled_xshift_prob,sampled_xshift_prob_for_eval,\
            sampled_xy_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
            sampled_policy_prob_eval,sampled_eval_policy_prob_eval, \
            sampled_xshift_prob_eval,mean,std,pdf2):
    
    
    
    ips_loss = []
    snips_loss = []
    ips_r_loss = []
    
    ips_loss_logging = []
    snips_loss_logging = []
    ips_r_loss_logging = []
    
    ips_loss_xshift = []
    snips_loss_xshift = []
    ips_r_loss_xshift = []
    
    ips_loss_both = []
    snips_loss_both = []
    ips_r_loss_both = []
    
    
    for i in range(10):
        eval_action = np.zeros(len(sampled_action))
        eval_action_prob = np.zeros(len(sampled_action))
        total_reward = 0
        for j in range(len(sampled_xy)):
            action, prob = take_action(sampled_ture_label_eval[i], eval_policy_dict)
            if action == sampled_ture_label_eval[i]:
                total_reward += 1
        total_reward = total_reward/len(sampled_xy)
        
        clf_xshift = p_xshift(sampled_xy,sampled_xy_eval)
        clf_logging = loggingpolicy(sampled_xy,sampled_action)
        
        logging = clf_logging.predict_proba(sampled_xy)[np.arange(len(sampled_xy)),np.array(sampled_action,np.int)]
        
        xshift_logging = clf_xshift.predict_proba(sampled_xy)
        r = xshift_logging[:,1]/xshift_logging[:,0]
        
        
        sampled_xshift_prob_eval = sampled_xshift_prob_for_eval
        
#         know all
        ips = sampled_eval_policy_prob/sampled_policy_prob * sampled_reward
        snips = sampled_eval_policy_prob/sampled_policy_prob * sampled_reward / np.mean(sampled_eval_policy_prob/sampled_policy_prob)
        ips_r = sampled_eval_policy_prob/sampled_policy_prob * (sampled_xshift_prob_eval/sampled_xshift_prob) * sampled_reward
        
        ips = sum(ips)/len(sampled_xy)
        snips = sum(snips)/len(sampled_xy)
        ips_r = sum(ips_r)/len(sampled_xy)
        
        ips_loss.append((ips-total_reward)**2)
        ips_r_loss.append((ips_r-total_reward)**2)
        snips_loss.append((snips-total_reward)**2)
        
        
#         unknow logging
        ips = sampled_eval_policy_prob/logging * sampled_reward
        snips = sampled_eval_policy_prob/logging * sampled_reward / np.mean(sampled_eval_policy_prob/logging)
        ips_r = sampled_eval_policy_prob/logging* (sampled_xshift_prob_eval/sampled_xshift_prob) * sampled_reward
        
        ips = sum(ips)/len(sampled_xy)
        snips = sum(snips)/len(sampled_xy)
        ips_r = sum(ips_r)/len(sampled_xy)
        
        ips_loss_logging.append((ips-total_reward)**2)
        ips_r_loss_logging.append((ips_r-total_reward)**2)
        snips_loss_logging.append((snips-total_reward)**2)
        
#         unknow xshift
        ips = sampled_eval_policy_prob/sampled_policy_prob * sampled_reward
        snips = sampled_eval_policy_prob/sampled_policy_prob * sampled_reward / np.mean(sampled_eval_policy_prob/sampled_policy_prob)
        ips_r = sampled_eval_policy_prob/sampled_policy_prob* r * sampled_reward
        
        ips = sum(ips)/len(sampled_xy)
        snips = sum(snips)/len(sampled_xy)
        ips_r = sum(ips_r)/len(sampled_xy)
        
        ips_loss_xshift.append((ips-total_reward)**2)
        ips_r_loss_xshift.append((ips_r-total_reward)**2)
        snips_loss_xshift.append((snips-total_reward)**2)
        
                
#         unknow both
        ips = sampled_eval_policy_prob/logging * sampled_reward
        snips = sampled_eval_policy_prob/logging * sampled_reward / np.mean(sampled_eval_policy_prob/logging)
        
        ips_r = sampled_eval_policy_prob/logging* r * sampled_reward
        ips = sum(ips)/len(sampled_xy)
        snips = sum(snips)/len(sampled_xy)
        ips_r = sum(ips_r)/len(sampled_xy)
        
        ips_loss_both.append((ips-total_reward)**2)
        ips_r_loss_both.append((ips_r-total_reward)**2)
        snips_loss_both.append((snips-total_reward)**2)
        
        
    return np.mean(ips_loss),np.mean(snips_loss),np.mean(ips_r_loss),\
            np.mean(ips_loss_logging),np.mean(snips_loss_logging),np.mean(ips_r_loss_logging),\
            np.mean(ips_loss_xshift),np.mean(snips_loss_xshift),np.mean(ips_r_loss_xshift),\
            np.mean(ips_loss_both),np.mean(snips_loss_both),np.mean(ips_r_loss_both)
        
        
            
        
        
        
        
        
        
        





In [26]:
def vis_gaussian(mean,cov):
    x, y = np.mgrid[0:1:.001, 0:1:.001]
    pos = np.dstack((x, y))
    rv = multivariate_normal(mean, cov)
#     fig2 = plt.figure()
#     ax2 = fig2.add_subplot(111)
    plt.contourf(x, y, rv.pdf(pos))
    plt.colorbar()
    plt.show()

In [27]:
# policy_dict = {
#     0:[0.9,0.1/3,0.1/3,0.1/3],
#     1:[0.2/3,0.8,0.2/3,0.2/3],
#     2:[0.3/3,0.3/3,0.7,0.7/3],
#     3:[0.4/3,0.4/3,0.4/3,0.6],
# }



# eval_policy_dict = {
#     0:[0.4,0.6/3,0.6/3,0.6/3],
#     1:[0.5/3,0.5,0.5/3,0.5/3],
#     2:[0.1/3,0.1/3,0.9,0.1/3],
#     3:[0.7/3,0.7/3,0.7/3,0.3],
# }
# # a = 0.8
# # b = 20
# size = 5000

# mean_logging = [0.,0.8]
# covariance_logging = [[0.1,0],[0,0.1]]

# mean_eval = [0.5,0.5]
# covariance_eval = [[10,0],[0,10]]
# vis_gaussian(mean_logging,covariance_logging)


In [28]:
# for m_i in range(len(mean_logging_list)):
#     for v_i in range(len(covariance_logging_list)):
#         mean_logging = mean_logging_list[m_i]
#         covariance_logging = covariance_logging_list[v_i]
#         print('mean,',mean_logging)
#         print('cov,',covariance_logging)
#         vis_gaussian(mean_logging,covariance_logging)
        

In [29]:
# sample_xy, action, reward, weight



In [30]:
from ope_v3 import *


class WEIGHT_DATA_SET(data.Dataset):
    '''
    dataset class with instance weight 
    '''
    def __init__(self, data,reward, weights):
        '''
        weights are same dimensional witi original data
        '''
        self.data = torch.tensor(data)
        self.reward = torch.tensor(reward)
        self.weights = torch.tensor(weights)

    def __getitem__(self, index):
        img = self.data[index] 
        target = self.reward[index]
        weight = self.weights[index]
        return img, target, weight

    def __len__(self):
        return len(self.data)

# r is xshift weight
# w is policy shift weight
def train_robust_regression(feature,action,reward,eval_xshift, logging_xshift,eval_policy,logging_policy,
                           dm=False,xshift_known = True, policy_known = True,use_xshift = True):
#     print('train models with true density ratios')
    Myy_robust_list = []
    Myx_robust_list = []
    model_robust_list = []
    for i in range(n_class):
        ## generate training set for action i

        idx = (action == i)


        data_i = feature[idx]
        reward_i = reward[idx]
        eval_xshift_i = eval_xshift[idx]
        logging_xshift_i = logging_xshift[idx]
        eval_policy_i = eval_policy[idx]
        logging_policy_i = logging_policy[idx]

        train_size = len(data_i)

        if xshift_known:
            r = logging_xshift_i/eval_xshift_i
        if policy_known:
            w = logging_policy_i/eval_policy_i
        if use_xshift:
            w = r * w
        
        if dm:
            w = torch.ones(len(data_i))

        weight_st = my_bound(w)
        weighted_train = WEIGHT_DATA_SET(data_i,reward_i, w)

        train_model = Net()

        validate_size = int(0.1*train_size)

        validate_loader = data.DataLoader(data.Subset(weighted_train, range(0, validate_size)),
            batch_size=32, shuffle=True)
        # 10% validation set
        train_loader = data.DataLoader(data.Subset(weighted_train, range(validate_size, train_size)),
            batch_size=32, shuffle=True, )

        train_model, Myy, Myx, _, _, _ = train_validate_test(args,args.lr_robust, args.epochs_training, "regression", 'cpu', 'False', train_model,
            train_loader, None , validate_loader, 4, 0.000, d = n_dim,testflag = False)

        model_robust_list.append(train_model)
        Myy_robust_list.append(Myy)
        Myx_robust_list.append(Myx)
    return model_robust_list, Myy_robust_list, Myx_robust_list




In [31]:
# model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(sampled_xy,sampled_action,sampled_reward,sampled_xshift_prob_for_eval
#                         ,sampled_xshift_prob,sampled_eval_policy_prob,sampled_policy_prob,
#                            dm=False,xshift_known = True, policy_known = True,use_xshift = True)

In [32]:
def predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,sampled_xy,sampled_action,sampled_xshift_prob_for_eval
                        ,sampled_xshift_prob,sampled_eval_policy_prob,sampled_policy_prob,
                           dm = False,xshift_known = True, policy_known = True,use_xshift = True):
    
    if xshift_known:
        r = sampled_xshift_prob/sampled_xshift_prob_for_eval
        
    if policy_known:
        w = sampled_policy_prob/sampled_eval_policy_prob
    
    if use_xshift:
        w = r*w
    if dm:
        w = torch.tensor([1])
    
    weight_st = my_bound(w)
    
    output = model_robust_list[int(sampled_action)](torch.tensor(sampled_xy).unsqueeze(0))

    Myy = Myy_robust_list[int(sampled_action)]
    Myx = Myx_robust_list[int(sampled_action)]
    meanY, varY = ru.predict_regression(torch.tensor(weight_st), Myy, Myx, output, mean0, var0)
    return meanY.detach().numpy()
        
        
        

In [33]:
def evaluate_robust(sampled_xy, sampled_action, sampled_reward, sampled_ture_label, \
        sampled_policy_prob,sampled_eval_policy_prob, \
        sampled_xshift_prob,sampled_xshift_prob_for_eval,\
        sampled_xy_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
        sampled_policy_prob_eval,sampled_eval_policy_prob_eval, \
        sampled_xshift_prob_eval,sampled_xshift_prob_for_logging,dm = False,use_xshift = True):
    
    model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(sampled_xy,sampled_action,sampled_reward,sampled_xshift_prob_for_eval
                        ,sampled_xshift_prob,sampled_eval_policy_prob,sampled_policy_prob,
                           dm=dm, use_xshift = use_xshift)
    
#     sample action 
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    for t in range(1):
        regression_list = []
        triple_r_list = []
        triple_list = []
        total_reward = 0
        for i in range(len(sampled_xy_eval)):
            action, prob = take_action(sampled_ture_label_eval[i], eval_policy_dict)
            if action == sampled_ture_label_eval[i]:
                total_reward += 1
            tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,sampled_xy_eval[i],
                               action,sampled_xshift_prob_eval[i]
                        ,sampled_xshift_prob_for_logging[i],prob,policy_dict[sampled_ture_label_eval[i]][action],dm = dm,use_xshift = use_xshift)
            regression_list.append(tmp_eval)

            #     triple
            
            rw = my_bound(sampled_xshift_prob_for_eval[i]/sampled_xshift_prob[i] \
                          * sampled_eval_policy_prob[i]/sampled_policy_prob[i])
            w = my_bound(sampled_eval_policy_prob[i]/sampled_policy_prob[i])

            
            tmp = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,sampled_xy[i],
                           sampled_action[i],sampled_xshift_prob_for_eval[i]
                    ,sampled_xshift_prob[i],sampled_eval_policy_prob[i],sampled_policy_prob[i],dm = dm,use_xshift = use_xshift)

            triple_r_list.append(rw*(sampled_reward[i] - tmp) + tmp_eval)

            triple_list.append(w*(sampled_reward[i] - tmp) + tmp_eval)


        triple_r = np.mean(triple_r_list)
        triple = np.mean(triple_list)
        robust = np.mean(regression_list)
        GT = total_reward/len(triple_r_list)
        reg_loss.append((robust - GT)**2)
        triple_loss.append((triple - GT)**2)
        triple_r_loss.append((triple_r - GT)**2)
    
    return np.mean(reg_loss),np.mean(triple_loss),np.mean(triple_r_loss)



In [34]:
# evaluate_robust(sampled_xy, sampled_action, sampled_reward, sampled_ture_label, \
#         sampled_policy_prob,sampled_eval_policy_prob, \
#         sampled_xshift_prob,sampled_xshift_prob_for_eval,\
#         sampled_xy_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
#         sampled_policy_prob_eval,sampled_eval_policy_prob_eval, \
#         sampled_xshift_prob_eval,sampled_xshift_prob_for_logging,dm = False,use_xshift = False)

In [35]:
model_list = [
    'known ips',
    'known snips',
    'known ips-r',
    'unknown ips',
    'unknown snips',
    'unknown ips-r',
    'known dm',
    'known dmdr',
    'known dmdr-r',
    'known robust without xshift',
    'known triple without xshift',
    'known triple-r without xshift',
    'known robust',
    'known triple',
    'known triple-r',
    'unknown dm',
    'unknown dmdr',
    'unknown dmdr-r',
    'unknown robust without xshift',
    'unknown triple without xshift',
    'unknown triple-r without xshift',
    'unknown robust',
    'unknown triple',
    'unknown triple-r'
]
len(model_list)

24

In [ ]:


# policy_dict = {
#     0:[0.9,0.1/3,0.1/3,0.1/3],
#     1:[0.2/3,0.8,0.2/3,0.2/3],
#     2:[0.3/3,0.3/3,0.7,0.7/3],
#     3:[0.4/3,0.4/3,0.4/3,0.6],
# }



# eval_policy_dict = {
#     0:[0.4,0.6/3,0.6/3,0.6/3],
#     1:[0.5/3,0.5,0.5/3,0.5/3],
#     2:[0.1/3,0.1/3,0.9,0.1/3],
#     3:[0.7/3,0.7/3,0.7/3,0.3],
# }


# size = 5000

# mean_logging_list =[[0.6,0.6], [0.8,0.8]]
# covariance_logging_list = [[[0.2,0],[0,0.2]],[[0.4,0],[0,0.4]]]

# mean_eval = [0.5,0.5]
# covariance_eval = [[10,0],[0,10]]
# vis_gaussian(mean_logging,covariance_logging)

# policy
# xshift policy
# size

# p = [0.9,0.8,0.7,0.6,0.5,0.5,0.6,0.7,0.8,0.9]






p = []
for i in range(n_class):
    p.append(np.random.uniform(0.3,0.7))


p_eval = []
for i in range(n_class):
    p_eval.append(np.random.uniform(0.6,0.8))


# p_eval = [0.6,0.7,0.6,0.7,0.6,0.7,0.6,0.7,0.6,0.7]

policy_dict = {}
for i in range(n_class):
    policy_dict[i] = []
    for t in range(n_class):
        if t == i:
            policy_dict[i].append(p[i])
        else:
            policy_dict[i].append((1-p[i])/n_class)

eval_policy_dict = {}
for i in range(n_class):
    eval_policy_dict[i] = []
    for t in range(n_class):
        if t == i:
            eval_policy_dict[i].append(p_eval[i])
        else:
            eval_policy_dict[i].append((1-p_eval[i])/n_class)



# eval_policy_dict = {
#     0:[0.4,0.6/3,0.6/3,0.6/3],
#     1:[0.5/3,0.5,0.5/3,0.5/3],
#     2:[0.1/3,0.1/3,0.9,0.1/3],
#     3:[0.7/3,0.7/3,0.7/3,0.3],
# }
# a = 0.8
# b = 20
# size = 5000

# mean_logging = [0.,0.8]
# covariance_logging = [[0.1,0],[0,0.1]]

# mean_eval = [0.5,0.5]
# covariance_eval = [[10,0],[0,10]]
# vis_gaussian(mean_logging,covariance_logging)



logging_a_list = [0.5,0.8,1.2]
logging_b_list = [0.1,0.2,0.4]
size_list = [500,1000,5000]
logging_policy_list = [[0.3,0.7],[0.6,0.8]]
eval_policy_list = [[0.5,0.6],[0.4,0.9]]

# logging_a = 0.5
# logging_b = 0.2

eval_a = 1
eval_b = 0.01

size = 2000

for size in size_list:
    frame = {}
    for logging_a in logging_a_list:
        for logging_b in logging_b_list:
            for logging_policy in  logging_policy_list:
                for eval_policy in eval_policy_list:
                    
                    p = []
                    for i in range(n_class):
                        p.append(np.random.uniform(logging_policy[0],logging_policy[1]))
                    p_eval = []
                    for i in range(n_class):
                        p_eval.append(np.random.uniform(eval_policy[0],eval_policy[1]))

                    policy_dict = {}
                    for i in range(n_class):
                        policy_dict[i] = []
                        for t in range(n_class):
                            if t == i:
                                policy_dict[i].append(p[i])
                            else:
                                policy_dict[i].append((1-p[i])/n_class)

                    eval_policy_dict = {}
                    for i in range(n_class):
                        eval_policy_dict[i] = []
                        for t in range(n_class):
                            if t == i:
                                eval_policy_dict[i].append(p_eval[i])
                            else:
                                eval_policy_dict[i].append((1-p_eval[i])/n_class)
                    
                    ips = []
                    snips = []
                    ips_r = []

                    ips_logging = []
                    snips_logging = []
                    ips_r_logging = []

                    ips_xshift = []
                    snips_xshift = []
                    ips_r_xshift = []

                    ips_both = []
                    snips_both = []
                    ips_r_both = []

                    dm = []
                    drrobust = []
                    drrobust_r = []

                    robust_noxshift = []
                    triple_noxshift = []
                    triple_r_noxshift = []

                    robust = []
                    triple = []
                    triple_r = []

                    dm_unknown = []
                    drrobust_unknown = []
                    drrobust_r_unknown = []

                    robust_noxshift_unknown = []
                    triple_noxshift_unknown = []
                    triple_r_noxshift_unknown = []

                    robust_unknown = []
                    triple_unknown = []
                    triple_r_unknown = []






                    for i in range(10):
                    #     pdf = get_gaussian_distribution(xy,logging_a,1.2)
                        first_component, pca_model, pdf, mean_logging, covariance_logging = get_distribution(xy,logging_a, logging_b)
                        pdf_norm = pdf/sum(pdf)
                        first_component_eval, pca_model, pdf2, mean_eval, covariance_eval = get_distribution(xy,eval_a, eval_b,pca_model)
                    #     pdf2 = get_gaussian_distribution(xy,mean_eval,covariance_eval)
                        pdf2_norm = pdf2/sum(pdf2)
                        sampled_xy,sampled_component, sampled_action, sampled_reward, sampled_ture_label, \
                            sampled_policy_prob,sampled_eval_policy_prob, \
                            sampled_xshift_prob,sampled_xshift_prob_for_eval  = sample_logging_data(xy,first_component,label,size,pdf_norm,pdf2,mean_eval,covariance_eval)

                        sampled_xy_eval,sampled_component_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
                            sampled_policy_prob_eval,sampled_eval_policy_prob_eval, \
                            sampled_xshift_prob_eval,sampled_xshift_prob_for_logging = sample_data(xy,first_component,label,size,pdf,pdf2_norm,mean_logging,covariance_logging)
                    #     ips, snips, ips_r = [], [], []
                        res = evaluate(sampled_xy, sampled_action, sampled_reward, \
                                     sampled_ture_label, sampled_policy_prob,\
                                     sampled_eval_policy_prob, sampled_xshift_prob,sampled_xshift_prob_for_eval,\
                                    sampled_xy_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
                                    sampled_policy_prob_eval,sampled_eval_policy_prob_eval, \
                                    sampled_xshift_prob_eval,mean_eval,covariance_eval,pdf2)

                        ips.append(res[0])
                        snips.append(res[1])
                        ips_r.append(res[2])

                        ips_logging.append(res[3])
                        snips_logging.append(res[4])
                        ips_r_logging.append(res[5])

                        ips_xshift.append(res[6])
                        snips_xshift.append(res[7])
                        ips_r_xshift.append(res[8])

                        ips_both.append(res[9])
                        snips_both.append(res[10])
                        ips_r_both.append(res[11])



                        res = evaluate_robust(sampled_xy, sampled_action, sampled_reward, sampled_ture_label, \
                            sampled_policy_prob,sampled_eval_policy_prob, \
                            sampled_xshift_prob,sampled_xshift_prob_for_eval,\
                            sampled_xy_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
                            sampled_policy_prob_eval,sampled_eval_policy_prob_eval, \
                            sampled_xshift_prob_eval,sampled_xshift_prob_for_logging,dm = True,use_xshift = False)
                        dm.append(res[0])
                        drrobust.append(res[1])
                        drrobust_r.append(res[2])

                        res = evaluate_robust(sampled_xy, sampled_action, sampled_reward, sampled_ture_label, \
                            sampled_policy_prob,sampled_eval_policy_prob, \
                            sampled_xshift_prob,sampled_xshift_prob_for_eval,\
                            sampled_xy_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
                            sampled_policy_prob_eval,sampled_eval_policy_prob_eval, \
                            sampled_xshift_prob_eval,sampled_xshift_prob_for_logging,dm = False,use_xshift = False)

                        robust_noxshift.append(res[0])
                        triple_noxshift.append(res[1])
                        triple_r_noxshift.append(res[2])

                        res = evaluate_robust(sampled_xy, sampled_action, sampled_reward, sampled_ture_label, \
                            sampled_policy_prob,sampled_eval_policy_prob, \
                            sampled_xshift_prob,sampled_xshift_prob_for_eval,\
                            sampled_xy_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
                            sampled_policy_prob_eval,sampled_eval_policy_prob_eval, \
                            sampled_xshift_prob_eval,sampled_xshift_prob_for_logging,dm = False,use_xshift = True)

                        robust.append(res[0])
                        triple.append(res[1])
                        triple_r.append(res[2])


                        clf_xshift = p_xshift(sampled_xy,sampled_xy_eval)
                        clf_logging = loggingpolicy(sampled_xy,sampled_action)

                        sampled_policy_prob_pred = clf_logging.predict_proba(sampled_xy)[np.arange(len(sampled_xy)),np.array(sampled_action,np.int)]
                        sampled_xshift_prob_pred = clf_xshift.predict_proba(sampled_xy)[:,0]
                        sampled_xshift_prob_for_eval_pred= clf_xshift.predict_proba(sampled_xy)[:,1]

                        sampled_policy_prob_eval_pred = clf_logging.predict_proba(sampled_xy_eval)[np.arange(len(sampled_xy_eval)),np.array(sampled_action_eval,np.int)]
                        sampled_xshift_prob_eval_pred = clf_xshift.predict_proba(sampled_xy_eval)[:,1]
                        sampled_xshift_prob_for_logging_pred= clf_xshift.predict_proba(sampled_xy_eval)[:,0]

                    #                 unknown
                        res = evaluate_robust(sampled_xy, sampled_action, sampled_reward, sampled_ture_label, \
                            sampled_policy_prob_pred,sampled_eval_policy_prob, \
                            sampled_xshift_prob_pred,sampled_xshift_prob_for_eval_pred,\
                            sampled_xy_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
                            sampled_policy_prob_eval_pred,sampled_eval_policy_prob_eval, \
                            sampled_xshift_prob_eval_pred,sampled_xshift_prob_for_logging_pred,dm = True,use_xshift = False)
                        dm_unknown.append(res[0])
                        drrobust_unknown.append(res[1])
                        drrobust_r_unknown.append(res[2])

                        res = evaluate_robust(sampled_xy, sampled_action, sampled_reward, sampled_ture_label, \
                            sampled_policy_prob_pred,sampled_eval_policy_prob, \
                            sampled_xshift_prob_pred,sampled_xshift_prob_for_eval_pred,\
                            sampled_xy_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
                            sampled_policy_prob_eval_pred,sampled_eval_policy_prob_eval, \
                            sampled_xshift_prob_eval_pred,sampled_xshift_prob_for_logging_pred,dm = False,use_xshift = False)

                        robust_noxshift_unknown.append(res[0])
                        triple_noxshift_unknown.append(res[1])
                        triple_r_noxshift_unknown.append(res[2])

                        res = evaluate_robust(sampled_xy, sampled_action, sampled_reward, sampled_ture_label, \
                            sampled_policy_prob_pred,sampled_eval_policy_prob, \
                            sampled_xshift_prob_pred,sampled_xshift_prob_for_eval_pred,\
                            sampled_xy_eval, sampled_action_eval, sampled_reward_eval, sampled_ture_label_eval, \
                            sampled_policy_prob_eval_pred,sampled_eval_policy_prob_eval, \
                            sampled_xshift_prob_eval_pred,sampled_xshift_prob_for_logging_pred,dm = False,use_xshift = True)

                        robust_unknown.append(res[0])
                        triple_unknown.append(res[1])
                        triple_r_unknown.append(res[2])





                    # print('size,',size)
                    # print(mean_logging)
                    # print(covariance_logging)

                    policy_name = str(logging_a) + '_' + str(logging_b) + str(logging_policy) + '_' + str(eval_policy)

                    result_list = []


                    print('all known')
                    print(np.mean(ips))
                    result_list.append(np.mean(ips))
                    print(np.mean(snips))
                    result_list.append(np.mean(snips))
                    print(np.mean(ips_r))
                    result_list.append(np.mean(ips_r))

                    print('unknown all')
                    print(np.mean(ips_both))
                    result_list.append(np.mean(ips_both))
                    print(np.mean(snips_both))
                    result_list.append(np.mean(snips_both))
                    print(np.mean(ips_r_both))
                    result_list.append(np.mean(ips_r_both))

                    print('dmrobust')
                    print(np.mean(dm))
                    result_list.append(np.mean(dm))
                    print(np.mean(drrobust))
                    result_list.append(np.mean(drrobust))
                    print(np.mean(drrobust_r))
                    result_list.append(np.mean(drrobust_r))

                    print('robust without xshift')
                    print(np.mean(robust_noxshift))
                    result_list.append(np.mean(robust_noxshift))
                    print(np.mean(triple_noxshift))
                    result_list.append(np.mean(triple_noxshift))
                    print(np.mean(triple_r_noxshift))
                    result_list.append(np.mean(triple_r_noxshift))

                    print('robust with xshift')
                    print(np.mean(robust))
                    result_list.append(np.mean(robust))
                    print(np.mean(triple))
                    result_list.append(np.mean(triple))
                    print(np.mean(triple_r))
                    result_list.append(np.mean(triple_r))
                    #             unknown
                    print('dmrobust unknown')
                    print(np.mean(dm_unknown))
                    result_list.append(np.mean(dm_unknown))
                    print(np.mean(drrobust_unknown))
                    result_list.append(np.mean(drrobust_unknown))
                    print(np.mean(drrobust_r_unknown))
                    result_list.append(np.mean(drrobust_r_unknown))

                    print('robust without xshift unknown')
                    print(np.mean(robust_noxshift_unknown))
                    result_list.append(np.mean(robust_noxshift_unknown))
                    print(np.mean(triple_noxshift_unknown))
                    result_list.append(np.mean(triple_noxshift_unknown))
                    print(np.mean(triple_r_noxshift_unknown))
                    result_list.append(np.mean(triple_r_noxshift_unknown))

                    print('robust with xshift unknown')
                    print(np.mean(robust_unknown))
                    result_list.append(np.mean(robust_unknown))
                    print(np.mean(triple_unknown))
                    result_list.append(np.mean(triple_unknown))
                    print(np.mean(triple_r_unknown))
                    result_list.append(np.mean(triple_r_unknown))
                    
                    frame[policy_name] = result_list
                    
                    
                    




    frame = pd.DataFrame(frame).round(4)
    frame.to_csv('mnist_result/' + str(size)+'.csv',index = False)




#             vis_gaussian(mean_logging,covariance_logging)
#             plt.scatter(sampled_xy[:,0],sampled_xy[:,1],c = sampled_ture_label + 1)



            # for k in range(4):
            #     print(sum(sampled_ture_label==k))
            # for k in range(4):
            #     print(sum(sampled_ture_label_eval==k))
            # print(' ')
            # for k in range(4):
            #     print(sum(sampled_ture_label==k)/sum(sampled_ture_label_eval==k))




all known
0.002140568553970677
0.0017638866321592606
0.00218123796762678
unknown all
0.007056360358037922
0.03722808831321329
0.16475798845506828
dmrobust
0.02621805667156723
0.003044872653156839
0.0032005418233130244
robust without xshift
0.0039953709125689005
0.0021940873623251507
0.002197538725009692
robust with xshift
0.002986903355799381
0.002490913726581687
0.002673683052122352
dmrobust unknown
0.03675456010097809
0.021124821332907242
0.017492722967128456
robust without xshift unknown
0.004713497798058719
0.0024744533213408642
0.004939799524331091
robust with xshift unknown
0.0026971933836332103
0.0005156585693347116
0.0017875417163634231
all known
0.017003703218476534
0.016319910806814567
0.017162833188027277
unknown all
0.036378973043604594
0.05581470511905449
0.28105796913464665
dmrobust
0.0029389465454263577
0.0015747853260397939
0.0016099669355962114
robust without xshift
0.0014096617285288934
0.0010018277901393978
0.0008975467276487985
robust with xshift
0.00650324309148583

all known
0.01984509742221468
0.018401933862607113
0.019929339925517386
unknown all
0.02968337994792894
0.04569401532078259
0.227335857650025
dmrobust
0.004867921933012416
0.0031283897194188345
0.003208118066872612
robust without xshift
0.02290726614865022
0.004538135402989379
0.003863476840473789
robust with xshift
0.01825178310158064
0.0036567779456888653
0.0036864189017345754
dmrobust unknown
0.006932541097469808
0.0015457321453606863
0.004310968154434231
robust without xshift unknown
0.03623635001743929
0.009662222004163296
0.011001400386139022
robust with xshift unknown
0.0062693027123193355
0.0013990931089864563
0.004466727215044557
all known
0.0017643249040687607
0.0022884032618228833
0.0017406856417591164
unknown all
0.014649002453269038
0.09735819277546087
0.17796032604942388
dmrobust
0.003364839599267186
0.002393792085510777
0.0023815041644606448
robust without xshift
0.0053071408562851185
0.005053921332857102
0.004904884705813026
robust with xshift
0.0037939288228060358
0.00

all known
0.0020407529375294216
0.00275717392798689
0.002101967700323795
unknown all
0.02458560197981865
0.07050589225833609
0.20301030687833926
dmrobust
0.00396170161892619
0.0007505482946729564
0.0007703513951056373
robust without xshift
0.0023050908613141165
0.0017345827682470893
0.0017291985257711212
robust with xshift
0.002338191004968856
0.0007793863371596447
0.0007873047967097849
dmrobust unknown
0.0049417204700527035
0.008636297506064563
0.012848042461186116
robust without xshift unknown
0.002564583839350791
0.011337102185994447
0.018308453377976493
robust with xshift unknown
0.0030696869512464543
0.008917177878471988
0.011786985818481376
all known
0.011909727767677219
0.01164118542407621
0.01182055636906279
unknown all
0.026038787461710754
0.06122020699839899
0.2644025679880587
dmrobust
0.0035419510654969995
0.003213096940294244
0.0032206113794791677
robust without xshift
0.003691459398817812
0.0011126611633450985
0.001082210999754371
robust with xshift
0.013959587880516374
0.

all known
0.008366394679661325
0.008321636103596847
0.008521154005144974
unknown all
0.1366804327700178
0.023809293855550473
0.3880609476926452
dmrobust
0.00330131681147025
0.0008952231168592127
0.0008015064665524297
robust without xshift
0.0030419418262056692
0.0008459214422259125
0.0009033091550657464
robust with xshift
0.003044195070600766
0.0006191574188751683
0.0005645422647479233
dmrobust unknown
0.001462855656263266
0.015468707736128736
0.022853197391936555
robust without xshift unknown
0.0034492845555153502
0.015251142205091679
0.022506043486655802
robust with xshift unknown
0.001489428164293679
0.014296756809398473
0.018538362462891193
all known
0.001573313644135508
0.0015241312326878783
0.001676357952291432
unknown all
0.08111560736984687
0.02914218432346975
0.30088312418161156
dmrobust
0.001831802781573155
0.0003657571102946052
0.0003037251182818012
robust without xshift
0.002276906313767347
0.0005722076313302034
0.00036257873005855165
robust with xshift
0.002907458058790100

all known
0.000977243913105629
0.000946601513190728
0.000962735677813238
unknown all
0.061995387116801914
0.027098439914750173
0.20127150122186155
dmrobust
0.0023557491862028477
0.0006999949532930586
0.0006933930700605996
robust without xshift
0.0010841141626664092
0.00036305442769578815
0.00036886866491401106
robust with xshift
0.002372898570984506
0.000249917013743211
0.000303515660080675
dmrobust unknown
0.0010899919556210567
0.0092828908857886
0.014225802384047442
robust without xshift unknown
0.001754461867250198
0.009453707139919946
0.015253733791562918
robust with xshift unknown
0.0010624599385736905
0.009087382099646978
0.01390069585057673
all known
0.010961017813199903
0.011002748047730266
0.011047508234905684
unknown all
0.10185828906485664
0.03163268346153799
0.3122605823678454
dmrobust
0.003459111886461
0.0002025247579688204
0.00023069895199773557
robust without xshift
0.0035348625591380633
0.0003446720121525116
0.0003095831395290293
robust with xshift
0.0033576930075624157

all known
0.010753138193246877
0.009734451151686715
0.010731897866974085
unknown all
0.11500756263726788
0.0266500874702744
0.3595091649300094
dmrobust
0.00665722110036693
0.000396326834456022
0.00044787064308445397
robust without xshift
0.004716341732207365
0.0004937318446013373
0.0005983495176485407
robust with xshift
0.003068716173559633
0.00044866862212114106
0.0004490352983120988
dmrobust unknown
0.0028909513707712086
0.009126667343169486
0.019446446107233485
robust without xshift unknown
0.002952760181151893
0.00873909152286726
0.01769263397660526
robust with xshift unknown
0.003696423949532318
0.008273165860691515
0.018398599287397864
all known
0.0015085851472601529
0.001459520225697503
0.0015084724716081385
unknown all
0.1403253863122146
0.03165620370704676
0.47546730879839705
dmrobust
0.007446554139961145
0.0015743436873735081
0.0015966801137502439
robust without xshift
0.002521095470087438
0.0018852687783682115
0.0017572260452733044
robust with xshift
0.003136826712817667
0.0

all known
0.0008650689228878582
0.0008355560409761814
0.0008975708273890476
unknown all
0.21896513196424888
0.0026330398711259763
0.3592537881093201
dmrobust
0.0036196858718645112
0.0004605340643055838
0.00017210314011802949
robust without xshift
0.0036286366967246777
0.0003104916880886337
0.00012577735577004752
robust with xshift
0.004064586998584626
0.0004087172996821324
0.00014391229800410913
dmrobust unknown
0.00390142378628619
0.08264826974617209
0.11049068902314887
robust without xshift unknown
0.00507991379778725
0.042113472862446724
0.050092673604154724
robust with xshift unknown
0.0018912782342756322
0.04735121429633041
0.053933674041616905
all known
0.00963946007240134
0.009457138936020782
0.009755871699663507
unknown all
0.21922344142882041
0.010620047799691874
0.3567835275554977
dmrobust
0.001944426475520205
0.00023851296960569418
0.00013552679172522874
robust without xshift
0.004480235059135895
0.00019421896181495688
0.00011287988291494614
robust with xshift
0.003257299600

all known
0.02177606890001857
0.021638322286696248
0.021583680495631807
unknown all
0.27330300797909957
0.0217297971102952
0.34179895518690334
dmrobust
0.0024764055818989983
6.880961267750833e-05
7.47595740845273e-05
robust without xshift
0.0016012760862855167
0.00010281817952457894
8.58719623599622e-05
robust with xshift
0.0012134674049287652
6.304591083037063e-05
6.659227682436438e-05
dmrobust unknown
0.002362504808636341
0.07523681096270371
0.07924327823767577
robust without xshift unknown
0.0016992321709162947
0.039623479552302034
0.04133287841794815
robust with xshift unknown
0.0016672465613171619
0.03891960376865552
0.03973208675579212
all known
0.0008322530170184299
0.0008540572380106862
0.0008007766684311058
unknown all
0.12587488520221035
0.013064402153324097
0.18033657756882987
dmrobust
0.001431632530516672
3.590917093701519e-05
5.198868427046925e-05
robust without xshift
0.0008342668812301873
4.259845380440116e-05
4.027732771340727e-05
robust with xshift
0.001199070063621017